In [2]:
## Libraries 
import re
from Bio import Entrez, SeqIO
# from packages import orf_scan_analyze, query_proteins, blast_proteins
import numpy as np
import random
import time
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from Bio.Blast import NCBIWWW, NCBIXML
# import random
# import time


### Fetch Data & Explore Data Objects

In [3]:
Entrez.email = "neel.mehtani@gmail.com"
handle = Entrez.einfo()

record = Entrez.read(handle)
handle.close()

record.keys()

dict_keys(['DbList'])

In [4]:
print(record["DbList"])

['pubmed', 'protein', 'nuccore', 'ipg', 'nucleotide', 'structure', 'genome', 'annotinfo', 'assembly', 'bioproject', 'biosample', 'blastdbinfo', 'books', 'cdd', 'clinvar', 'gap', 'gapplus', 'grasp', 'dbvar', 'gene', 'gds', 'geoprofiles', 'homologene', 'medgen', 'mesh', 'ncbisearch', 'nlmcatalog', 'omim', 'orgtrack', 'pmc', 'popset', 'proteinclusters', 'pcassay', 'protfam', 'biosystems', 'pccompound', 'pcsubstance', 'seqannot', 'snp', 'sra', 'taxonomy', 'biocollections', 'gtr']


In [5]:
handle = Entrez.esearch(db="nucleotide", term="Brassica napus")

record = Entrez.read(handle)
handle.close()

record.keys()

dict_keys(['Count', 'RetMax', 'RetStart', 'IdList', 'TranslationSet', 'TranslationStack', 'QueryTranslation'])

In [6]:
print(record["IdList"][:5])

['937575234', '2194135529', '2193981524', '2191950016', '2191950006']


In [11]:
## https://www.ncbi.nlm.nih.gov/Traces/wgs/?display=contigs&page=1
Entrez.email = "neel.mehtani@gmail.com"
handle = Entrez.efetch(db="nucleotide", rettype="gb", id="QGKT01000001")
record = SeqIO.read(handle, "genbank")
seq = record.seq
print("First 100 sequence nts:\n", seq[:100])

First 100 sequence nts:
 TCAACCACAAAGTCAAAAAACCAAATCCATCCTTTTAATTTTAAAAAAAAGTATGATTATTATTTTAGATTTTTTTCTAATCTATCTCTTTATTGCTTCT


In [8]:
print("Length of sample genome sequence: {}".format(len(record.seq)/1000000, "Mb"))

Length of sample genome sequence: 0.373899


### Scan genomic strands for potential ORFs longer than 50 codons & translate to proteins

In [33]:
## Store sequence from sequence object
seq = record.seq

## Parameters
table = 11
min_pro_len = 50

## Lists for ORF protein details
orf_proteins, protein_wts, protein_lens = [], [], []

## Start scan of both genomic strands
for strand, nuc in [(+1, seq), (-1, seq.reverse_complement())]:
    
    ## Scan each possible frame
    for frame in range(3):
        
        ## Transalte the frame
        for pro in nuc[frame:].translate(table).split("*"):
            
            if len(pro) >= min_pro_len:
                
                ## Run a protein "param" analysis -- returns molecular weight/amino acid freqs etc.
                analysis = ProteinAnalysis(str(pro))
                
                ## Add protein details 
                orf_proteins.append(pro)
                protein_wts.append(round(analysis.molecular_weight(), 3))
                protein_lens.append(len(pro))
                
                ## Output to screen
                print("%s...%s - length %i, strand %i, frame %i" % (pro[:30], pro[-3:], len(pro), strand, frame))


## ORF Protein details
protein_wts, protein_lens = np.array(protein_wts), np.array(protein_lens)
orf_proteins = np.array(orf_proteins, dtype='object')

## Output 5 protein molar masses to screen
for i in range(5):
    print("Protein " + str(orf_proteins[i]) + ":  " + str(protein_wts[i]) + "\n\n")

## Randomized selection for 5 proteins
query_protein_idxs = random.sample(list(np.where(protein_lens < 1000)[0]), 5)

## Determine query proteins with selected idxs
orf_proteins_arr = np.array(orf_proteins, dtype='object')
query_proteins = orf_proteins_arr[query_protein_idxs]

### BLAST 5 Proteins of length <= 1000 amino acids

In [33]:
query_ct = 1 ## Counter for tagging results
for q in query_proteins:
    
    ## Run BLAST & store returned XML file results 
    out_file = open("blast_{}.xml".format(query_ct), "w") 
    blast_handle = NCBIWWW.qblast("blastp", "nr", q)

    out_file.write(blast_handle.read())
    out_file.close()

    blast_handle.close()
    
    ## Open BLAST XML results to write to custom txt file output
    result_handle = open("blast_{}.xml".format(query_ct))
    blast_records = NCBIXML.read(result_handle)
    
    save_file = open("BLAST_query_{}_result_log".format(query_ct), "w")
    save_file.write("BLAST Query #{} Results -- Sequence Example\n".format(query_ct) + "--------------------------" + "\n\n")

    ## Set E-Value threshold for sequences we want to write to output
    e_value_thresh = 0.25
    for alignment in blast_records.alignments:
        for hsp in alignment.hsps:
            if hsp.expect < e_value_thresh:
                save_file.write("sequence: {} \n".format(alignment.title))
                save_file.write("length: {} \n".format(alignment.length))
                save_file.write("E Value: {} \n".format(hsp.expect))
                save_file.write("{} ...  \n".format(hsp.query[0:20]))
                save_file.write("{} ...  \n".format(hsp.match[0:20]))
                save_file.write("{} ...  \n".format(hsp.sbjct[0:20]))
                save_file.write("\n\n")
            else:
                save_file.write("NULL RESULT -- No hits returned!!")
    
    result_handle.close()
    query_ct += 1

In [35]:
# def orf_scan_analyze(seq, log_dir, min_pro_len=50, table=11):
    
#     ## Lists for ORF protein details
#     orf_proteins = []
#     protein_wts = []
#     protein_lens = []
    
#     ## Start scan of both genomic strands
#     for strand, nuc in [(+1, seq), (-1, seq.reverse_complement())]:
    
#         ## Scan each possible frame
#         for frame in range(3):

#             ## Transalte the frame
#             for pro in nuc[frame:].translate(table).split("*"):

#                 if len(pro) >= min_pro_len:

#                     ## Run a protein "param" analysis -- returns molecular weight/amino acid freqs etc.
#                     analysis = ProteinAnalysis(str(pro))

#                     ## Add protein details 
#                     orf_proteins.append(pro)
#                     protein_wts.append(round(analysis.molecular_weight(), 3))
#                     protein_lens.append(len(pro))
                    
#                     # Output to screen
#                     # print("%s...%s - length %i, strand %i, frame %i" % (pro[:30], pro[-3:], len(pro), strand, frame))

#     protein_wts = np.array(protein_wts)
#     protein_lens = np.array(protein_lens)
#     orf_proteins_arr = np.array(orf_proteins, dtype='object')

#     save_file_path = log_dir + "/" + "protein_weights" + '.txt'
#     save_file = open(save_file_path, "w")
#     save_file.write("Protein Molar Mass Results\n" + "--------------------------------------------" + "\n\n")
    
#     for i in range(len(protein_wts)):
#         save_file.write("Protein " + str(orf_proteins[i]) + ":  " + str(protein_wts[i]) + "\n\n")

#     return orf_proteins_arr, protein_lens, query_protein_idxs

# def query_proteins(orf_proteins, protein_lens):
    
    
#     query_protein_idxs = random.sample(list(np.where(protein_lens < 1000)[0]), 5)
#     query_proteins = orf_proteins[query_protein_idxs]

    
#     return query_proteins

# def blast_proteins(query_proteins, log_dir, e_value_thresh=0.04):
    
#     Entrez.email = "neel.mehtani@gmail.com"
#     query_ct = 1
    
#     for q in query_proteins:
        
#         out_file_path = log_dir + "/" + "blast_{}.xml".format(query_ct) 
#         out_file = open(out_file_path, "w") 
        
#         blast_handle = NCBIWWW.qblast("blastp", "nr", q)
#         out_file.write(blast_handle.read())
#         out_file.close()
#         blast_handle.close()

#         result_handle = open(out_file_path)
#         blast_records = NCBIXML.read(result_handle)
        
#         save_file_path = log_dir + "/" + "BLAST_query_{}_result_log".format(query_ct) + '.txt'
#         save_file = open(save_file_path, "w")
#         save_file.write("BLAST Query #{} Results -- Sequence Example\n".format(query_ct) + "----------------------------------------" + "\n\n")
        
#         for alignment in blast_records.alignments:

#             for hsp in alignment.hsps:
#                 if hsp.expect < e_value_thresh:

#     #                 content = ["sequence: {} \n".format(alignment.title), "length: {} \n".format(alignment.length),
#     #                            "E Value: {} \n".format(hsp.expect), "{} ...  \n".format(hsp.query[0:20]), "{} ...  \n".format(hsp.match[0:20]), 
#     #                            "{} ...  \n".format(hsp.sbjct[0:20]), "\n\n"]

#                     save_file.write("sequence: {} \n".format(alignment.title))
#                     save_file.write("length: {} \n".format(alignment.length))
#                     save_file.write("E Value: {} \n".format(hsp.expect))
#                     save_file.write("{} ...  \n".format(hsp.query[0:20]))
#                     save_file.write("{} ...  \n".format(hsp.match[0:20]))
#                     save_file.write("{} ...  \n".format(hsp.sbjct[0:20]))
#                     save_file.write("\n\n")
#                 else:
#                     save_file.write("NULL RESULT -- No hits returned!!")

#         result_handle.close()
#         query_ct += 1

#     return 

In [2]:
# if __name__ == "__main__":    
    
#     ## Set seed
#     random.seed(time.time())

#     ## Use Entrez for NCBI querying
#     Entrez.email = "neel.mehtani@gmail.com"

#     ## Handle to extract sequence data
#     handle = Entrez.efetch(db="nucleotide", rettype="gb", id="QGKT01000001")
#     record = SeqIO.read(handle, "genbank")
#     handle.close()

#     ## Extract sequence information from seq object
#     seq = record.seq

#     ## Parameters 
#     min_pro_len = 50 # ORF threshold
#     table = 11 # translation reference table
#     threshold = 0.3 # BLAST E-value threshold for output logs

#     ## Output Directory
#     log_dir = "../data"

#     ## ORF Scanning, codon translation, and molar mass calculation
#     orf_proteins_arr, protein_lens = orf_scan_analyze(seq, log_dir, table=table)

#     ## Query Protein Indexing
#     q_proteins = query_proteins(orf_proteins_arr, protein_lens)

#     ## BLAST Analysis
#     blast_proteins(q_proteins, log_dir, e_value_thresh=threshold)

/Applications/anaconda3/lib/python3.6/site-packages/Bio/Seq.py:2983: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning,
